# Azure OpenAI with lanchain SQLAgents
This code demonstrates how to use langchain SQL Agents with Azure OpenAI to produce structured output.

In [ ]:
! pip install pymssql --Uq
! pip install openai --Uq
! pip install python-dotenv --Uq
! pip install langchain --Uq
! pip install openai[datalib] --upgrade

## Import required libraries and environment variables

In [ ]:
# Import required libraries  
import os  
import openai  
from dotenv import load_dotenv  

# Configure environment variables  
load_dotenv(override=True)  
# OpenAI init
openai.api_type = "azure"  
openai.api_version = os.getenv("OPENAI_API_VERSION")  
sql_user_name = os.getenv("SQL_USER_NAME")  
sql_user_password = os.getenv("SQL_USER_PASSWORD")  
sql_server_name = os.getenv("SQL_SERVER_NAME")  
sql_db_name = os.getenv("SQL_DB_NAME")  



# Azure SQL Database
Query all applications deployments from Azure SQL Database and summarize by cloud type.
:warning: lanchain will try read metadata from all tables that user has access to. This will be very slow i.e. several minutes on any reasonable database.
for reasonable perfomance please make sure that database user has access only to tables that are required for the analysis.

Warning! as of today langchain does not seem always properly use TOP vs LIMIT for Microsoft SQL dialect. TBD to figure out how ensure it does.


In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import AzureChatOpenAI

from langchain.output_parsers import PydanticOutputParser
# To parse outputs and get structured data back
from pydantic import BaseModel, Field
from typing import List

openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_key = os.getenv("OPENAI_API_KEY")  
llm=AzureChatOpenAI(deployment_name="gpt-35-turbo-16k", openai_api_base= openai.api_base, openai_api_key=openai.api_key, temperature=0)

db = SQLDatabase.from_uri(f"mssql+pymssql://{sql_user_name}:{sql_user_password}@{sql_server_name}:1433/{sql_db_name}", view_support=True)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=10,
    verbose_iterations=True,
    use_query_checker=True, 
    return_intermediate_steps=True,
)

class CloudDeploymentsStat(BaseModel):
    cloudName: str = Field(description="Cloud for solution deployment.")
    deploymentsCount: str = Field(description="Number of deployments.")
    
class CloudDeploymentsStatList(BaseModel):
    items: List[CloudDeploymentsStat]

parser = PydanticOutputParser(pydantic_object=CloudDeploymentsStatList)
print(parser.get_format_instructions())

response=agent_executor.run("calculate number of deployments we have for each cloud ? Do not limit number of rows, when cloud is 'None' return cloud type as 'Not cloud'. "+parser.get_format_instructions())
 
print(response)
print(parser.parse(response))



